In [ ]:
kmeans = KMeans(n_clusters = 5, init='k-means++',max_iter = 20)
s = kmeans.fit(data_scaled)
s_centroids = s.cluster_centers_

plt.scatter(data_scaled[:,0], data_scaled[:,1], s=50, alpha=0.5)
plt.scatter(s_centroids[:, 0], s_centroids[:,1], c='red', s=50)
plt.show()

In [ ]:
# Dimentionality Reduction before training with Kmeans.
# Dimentionality Reduction is the technique used to identify the most influential features for machine learning
# The input data in this case contains just ten features. Hence, this may not be required
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

# Assigning the input frame to a simpler variable
df = ml_commits_numeric_na
# Creating the instance of Standard Scaler
scaler = StandardScaler()
# Transforming the input data using Standard Scaler
data_scaled = scaler.fit_transform(df)
# Creating the instance of PCA. PCA is a popular tool for the purpose of Dimentionality Reduction
pca = PCA()
# Processing the scaled data using PCA
pca.fit(data_scaled)
# Printing the variance ratio for each feature. FYI, each column of the frame represents a feature
pca.explained_variance_ratio_

In [ ]:
#Plotting the variance ratios for each feature
import matplotlib.pyplot as plt
plt.figure(figsize = (12,8))
plt.plot(range(1,7),pca.explained_variance_ratio_.cumsum(),marker = 'o',linestyle = '--')
plt.title('Explained Variance by Component')
plt.xlabel('Number of Components')
plt.ylabel('Cumulatve Explained Variance')

In [ ]:
# From the above plot, it is clear that first three features have major impact on variance. And the last five have no imapct at all.
# Hence, we can try Kmeans with first three features
pca = PCA(n_components = 6)
pca.fit(data_scaled)
scores_pca = pca.transform(data_scaled)
print(scores_pca.shape)

In [ ]:
# Training the input frame with PCA processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
df = ml_commits_nout_numeric
scaler = StandardScaler()
normalizer = Normalizer()
data_scaled = scaler.fit_transform(df)


pca = PCA(n_components = 6)
pca.fit(data_scaled)
scores_pca = pca.transform(data_scaled)
kmeans = KMeans(n_clusters = 5, init='random',max_iter = 10)
q = kmeans.fit(scores_pca)

plt.scatter(scores_pca[:,0], scores_pca[:,1], s=50, alpha=0.5)
plt.scatter(q_centroids[:, 0], q_centroids[:, 1], c='red', s=50)
plt.show()

In [ ]:
# Function to try out various input models
# Includes measurement of quality of Clustering through silhouette_scores
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

print(__doc__)

# X can be assigned to any input frame.
X = data_normalized
# If you want to try out Scaled frame, replace the above line with the following
# X = data_scaled
# For PCA frame, replace with the following line
# X = scores_pca
range_n_clusters = [2, 3, 4, 5, 6,8]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10,max_iter = 50,init = 'random')
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values =  sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    # In the below line, two features, X[:,4] and X[:,6] are selected for plotting
    # Those features can be replaced with any other two features.
    # Keep in mind that, the frame contains ten features, from X[:,0] to X[:,9]
    # In the case of PCA, there are only three features selected after the dimentionality reduction.
    # Hence, if the X is assigned to scores_pca, you can select from X[:,0] to X[:2]
    ax2.scatter(X[:, 4], X[:, 6], marker='.', s=150, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    # The centers should have same positions as features. 
    ax2.scatter(centers[:,4], centers[:,6], marker='o',
                c="red", alpha=1, s=300, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[4], c[6], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

In [ ]:
# Function to try out various input models
# This is same as the above except that the input is limited to just two features
# The puprose here is to understand how cluster behaves if we have same number of training features as plotted
# Includes measurement of quality of Clustering through silhouette_scores
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

print(__doc__)

# X can be assigned to any input frame.
X = data_normalized[:,[3,5]]
# If you want to try out Scaled frame, replace the above line with the following
# X = data_scaled[:,[3,5]]
# For PCA frame, replace with the following line
# X = scores_pca[:,[0,1]]
range_n_clusters = [2, 3, 4, 5, 6,8]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10,max_iter = 50,init = 'random')
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values =  sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    # Here, we can only plot X[:, 0], X[:, 1] as the training was done on just two features
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=150, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    # The centers should have same positions as features. 
    ax2.scatter(centers[:,0], centers[:,1], marker='o',
                c="red", alpha=1, s=300, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

In [ ]:
import scipy.stats as ss
data_scaled = scaler.fit_transform(ml_commits_all_na)
num_cols = len(scores_pca[0,:])
for i in range(num_cols):
    col = scores_pca[:,i]
    col_stats = ss.describe(col)
    print(col_stats)

In [ ]:
import elasticsearch
from elasticsearch_dsl import Search,Q
import pandas as pd
#p1[0],p1[1],p1[2]
es = p1[0]
es_ma_index = p1[1]
es_bl_index =p1[2]
# Using Elasticsearch DSL function to get the data of Commit index
#print(es,es_ma_index,es_bl_index)
blame_es_data = Search(using=es, index=es_bl_index)
        # Loading data into a dictionary
blame_dict = [hit.to_dict() for hit in blame_es_data.scan()]
print(len(blame_dict))
        # Using Elasticsearch DSL function to get the data of Blame index
commit_es_data = Search(using=es, index=es_ma_index)
        # Loading data into a dictionary
commit_dict = [hit.to_dict() for hit in commit_es_data.scan()]
print(len(commit_dict))
        # Creating pandas dataframe for commit data
commit_frame = pd.DataFrame(commit_dict)
        # Creating pandas dataframe for blame data
blame_frame = pd.DataFrame(blame_dict)
        # Getting the blame row count. If the frmae is empty, it means all the records are clean
blame_count = blame_frame.shape[0]